In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

In [ ]:
from gpax.kernels import RBFKernel
from gpax.gp import ExactGP, VariationalInferenceGP
from gpax.experiments import SimpleSinusoidal1d
from gpax.state import set_rng_key, get_rng_key

# Set the experiment data and key

In [ ]:
set_rng_key(1)

In [ ]:
experiment = SimpleSinusoidal1d()
x, y = experiment.default_dataset()
x_grid = experiment.get_dense_coordinates(ppd=100)

# Exact GP

In [ ]:
gp = ExactGP(kernel=RBFKernel(), x=x, y=y, y_std=None, hp_samples=123, gp_samples=456, verbose=1)
gp.fit()

In [ ]:
samples = gp.sample(x_grid)
mean, std = gp.predict(x_grid)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

plot_kwargs = {
    'linewidth': 0.0,
    'marker': 's',
    'ms': 1.0,
    'capthick': 0.3,
    'capsize': 2.0,
    'elinewidth': 0.3
}

for yy in samples["y"].mean(axis=1):
    ax.plot(x_grid, yy, color="grey", linewidth=0.5, alpha=0.4)

ax.errorbar(x, y, yerr=gp.y_std if gp.y_std is not None else None, color="black", zorder=3, **plot_kwargs)
ax.plot(x_grid, mean, "r-")
ax.fill_between(x_grid.squeeze(), mean - std, mean + std, color="red", alpha=0.5, linewidth=0, zorder=3)

plt.show()

# Simple acquisition function test

In [ ]:
from gpax.acquisition import UpperConfidenceBound, ExpectedImprovement

In [ ]:
bounds = np.array([-1.1, 1.1]).reshape(-1, 1)

## EI

In [ ]:
mc_samples = 1000
acqf = ExpectedImprovement(q=1, bounds=bounds, verbose=1)
x_star, y_star = acqf.optimize(gp, n=mc_samples, method="Halton")
print(x_star)
acqf = ExpectedImprovement(q=2, bounds=bounds, verbose=1)
x_star, y_star = acqf.optimize(gp, n=mc_samples, method="Halton")
print(x_star)

## UCB

In [ ]:
mc_samples = 1000
acqf = UpperConfidenceBound(beta=np.inf, q=1, bounds=bounds, verbose=1)
x_star, y_star = acqf.optimize(gp, n=mc_samples, method="Halton")
print(x_star)
acqf = UpperConfidenceBound(beta=np.inf, q=2, bounds=bounds, verbose=1)
x_star, y_star = acqf.optimize(gp, n=mc_samples, method="Halton")
print(x_star)

# VI GP

In [ ]:
gp = VariationalInferenceGP(kernel=RBFKernel(), x=x, y=y, y_std=None, num_steps=10000)
gp.fit()

In [ ]:
samples = gp.sample(x_grid)
mean, std = gp.predict(x_grid)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

plot_kwargs = {
    'linewidth': 0.0,
    'marker': 's',
    'ms': 1.0,
    'capthick': 0.3,
    'capsize': 2.0,
    'elinewidth': 0.3
}

for yy in samples["y"].squeeze():
    ax.plot(x_grid, yy, color="grey", linewidth=0.5, alpha=0.4)

ax.errorbar(gp.x, gp.y, yerr=gp.y_std if gp.y_std is not None else None, color="black", zorder=3, **plot_kwargs)
ax.plot(x_grid, mean, "r-")
ax.fill_between(x_grid.squeeze(), mean - std, mean + std, color="red", alpha=0.5, linewidth=0, zorder=3)

plt.show()